In [1]:
import pandas as pd
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import os
import seaborn as sns
import pandas as pd
from math import e,sqrt
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from scipy.optimize import leastsq
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import datetime
import statistics
import math
import os
import sys
import statistics
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")

from embfts.util.DataSetUtil import DataSetUtil
from embfts.util.StatisticsUtil import StatisticsUtil

from sklearn.svm import SVR
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
from scipy.stats import randint as sp_randint


In [3]:
data_set_util = DataSetUtil()
statistics_util = StatisticsUtil()

In [4]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [5]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/energydata_complete.csv', sep=',')
data = df.drop(labels=['date','rv1','rv2'], axis=1)
data = data_set_util.clean_dataset(data)
data = data_set_util.series_to_supervised_mimo(data, 1, 1)
data.head()

,Appliances(t-1),lights(t-1),T1(t-1),RH_1(t-1),T2(t-1),RH_2(t-1),T3(t-1),RH_3(t-1),T4(t-1),RH_4(t-1),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
1,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9
5,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.29,6.016667,734.0,92.0,5.333333,43.833333,4.8


In [6]:
# x,y=data.loc[:,:],data.loc[:,:]
# from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# # Get the number of inputs and outputs from the dataset
# df_trainx=x_train.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
# df_testx=x_test.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
# df_trainy=y_train.loc[:,'Appliances(t)':'Tdewpoint(t)']
# df_original=y_test.loc[:,'Appliances(t)':'Tdewpoint(t)']

# n_inputs, n_outputs = x.shape[1], y.shape[1]

In [7]:
def create_and_fit_model(X_train,y_train):
    #clf = MultiOutputRegressor(estimator=SVR(C=18.291137504308498, epsilon=0.7817324250849084))
    #clf = MultiOutputRegressor(estimator=SVR(C=12.623399352446802, epsilon=0.0007589726482352122))
    clf = MultiOutputRegressor(estimator=SVR(C=5.7642040130783005, epsilon=0.07067015766896201))
    clf.fit(X_train, y_train)
    return clf

In [8]:
def forecast_model(clf, X_test):
    forecast = clf.predict(X_test)
    return forecast

In [9]:
def sliding_window(data,n_windows,train_size):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            scaler = StandardScaler()

            
#             Xtrain = ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             ytrain = ttrain.loc[:,'Appliances(t)':'Tdewpoint(t)']
#             Xtest = ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             ytest = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)']
            
            Xtrain = scaler.fit_transform(ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)'])
            ytrain = scaler.fit_transform(ttrain.loc[:,'Appliances(t)':'Tdewpoint(t)'])
            Xtest = scaler.transform(ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)'])
            ytest = scaler.transform(ttest.loc[:,'Appliances(t)':'Tdewpoint(t)'])
            
            
            model = create_and_fit_model(Xtrain,ytrain)
            forecast = forecast_model(model, Xtest)
             
            df_forecats_columns = ['Appliances(t)', 'lights(t)', 'T1(t)', 'RH_1(t)', 'T2(t)', 'RH_2(t)',
                                   'T3(t)', 'RH_3(t)', 'T4(t)', 'RH_4(t)', 'T5(t)', 'RH_5(t)', 'T6(t)',
                                   'RH_6(t)', 'T7(t)', 'RH_7(t)', 'T8(t)', 'RH_8(t)', 'T9(t)', 'RH_9(t)',
                                   'T_out(t)', 'Press_mm_hg(t)', 'RH_out(t)', 'Windspeed(t)',
                                   'Visibility(t)', 'Tdewpoint(t)']
            
            columns = list(df_forecats_columns)
            
            prediction = scaler.inverse_transform(forecast) 
            #prediction = forecast
            df_forecast = pd.DataFrame(prediction,columns=columns)
            
            #ytest_metric = scaler.inverse_transform(y_test.reshape(y_test.shape[0], y_test.shape[2]))
            ytest_metric = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)'].values
            df_original = pd.DataFrame(ytest_metric,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

In [10]:
n_windows = 30
train_size = 0.75 

model_svr = sliding_window(data,n_windows,train_size)

--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 657


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 1314


...

/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 18396


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 19053


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


In [12]:
df_forecats_columns = ['Appliances(t)', 'lights(t)', 'T1(t)', 'RH_1(t)', 'T2(t)', 'RH_2(t)',
                                   'T3(t)', 'RH_3(t)', 'T4(t)', 'RH_4(t)', 'T5(t)', 'RH_5(t)', 'T6(t)',
                                   'RH_6(t)', 'T7(t)', 'RH_7(t)', 'T8(t)', 'RH_8(t)', 'T9(t)', 'RH_9(t)',
                                   'T_out(t)', 'Press_mm_hg(t)', 'RH_out(t)', 'Windspeed(t)',
                                   'Visibility(t)', 'Tdewpoint(t)']

columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

model_svr.to_csv (r'all_windows_svr_uci_appliances.csv', index = False, header=True)

measures = model_svr
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures = round(pd.DataFrame(final_result),3)

In [13]:
final_measures

,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,Appliances(t),86.812,58.108,74.747,-1.044000e+00,25.819,0.206,30.958,18.418,29.265,3.969000e+00,5.838,0.155
1,lights(t),7.064,5.123,inf,-1.108000e+00,87.473,inf,2.909,2.417,NaN,4.884000e+00,11.160,NaN
2,T1(t),0.702,0.565,2.675,-1.332000e+00,1.337,0.362,0.460,0.393,2.048,5.098000e+00,1.014,0.244
3,RH_1(t),2.054,1.664,4.095,-1.219000e+00,2.039,0.266,1.357,1.207,2.797,2.701000e+00,1.415,0.180
4,T2(t),1.043,0.849,4.144,-3.020000e-01,2.073,0.296,0.524,0.431,2.215,1.798000e+00,1.092,0.172
5,RH_2(t),2.203,1.822,4.583,-1.202000e+00,2.284,0.303,1.655,1.484,3.362,3.807000e+00,1.734,0.242
6,T3(t),0.725,0.604,2.676,-2.707000e+00,1.351,0.404,0.499,0.435,1.946,5.597000e+00,0.995,0.312
7,RH_3(t),1.499,1.235,3.106,-1.841000e+00,1.555,0.342,1.078,0.995,2.256,3.868000e+00,1.167,0.223
8,T4(t),0.858,0.729,3.630,-3.108000e+00,1.812,0.459,0.560,0.492,2.768,6.215000e+00,1.349,0.318
9,RH_4(t),1.903,1.570,3.889,-3.558000e+00,1.972,0.406,1.523,1.466,3.198,8.877000e+00,1.705,0.318


In [14]:
final_measures.to_csv (r'svr_uci_appliances_with_std.csv', index = False, header=True)

## RandomizedSearchCV

In [9]:
from scipy import stats
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
# Define the hyperparameter configuration space
hyperparameters = dict(
                     estimator__C=stats.uniform(0,50),
                     estimator__kernel=['poly', 'rbf','sigmoid'],
                     estimator__epsilon=stats.uniform(0,1))
clf=SVR()
model=MultiOutputRegressor(clf)
gsc = RandomizedSearchCV(model,hyperparameters,cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1,n_iter=10)

hyperparameters_tuning = gsc.fit(df_trainx, df_trainy)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

Best Parameters = {'estimator__C': 18.291137504308498, 'estimator__epsilon': 0.7817324250849084, 'estimator__kernel': 'poly'}


In [11]:
hyperparameters_tuning.best_estimator_

MultiOutputRegressor(estimator=SVR(C=18.291137504308498,
                                   epsilon=0.7817324250849084, kernel='poly'))

In [10]:
y_p=tuned_model.predict(df_testx)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,Appliances(t),lights(t),T1(t),RH_1(t),T2(t),RH_2(t),T3(t),RH_3(t),T4(t),RH_4(t),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
0,40.621778,0.778683,20.810488,35.810525,18.225685,39.097013,20.853483,36.490768,20.243530,34.450889,...,22.379189,39.871025,18.978586,39.180423,1.533144,768.200006,88.596059,2.302013,62.927985,0.000234
1,86.712550,8.052511,20.918058,49.861327,20.798448,45.714178,21.405625,47.600275,20.033286,43.360119,...,20.203847,47.947205,17.104066,44.498584,2.865403,754.283800,89.936213,3.215291,27.022790,1.491759
2,50.060563,0.780199,20.961698,35.807480,17.765461,39.859060,21.563194,35.428137,20.432932,33.294852,...,22.451710,38.916218,19.362971,38.123280,0.416406,765.526772,83.214669,2.142452,64.514835,-2.047346
3,52.382597,0.779913,21.943029,39.718090,22.522914,36.099070,22.674315,36.836052,21.940806,35.592235,...,21.938020,38.257497,20.418916,38.222911,9.095573,767.172191,70.845386,4.563243,39.635029,3.731308
4,66.780862,0.777655,20.160421,35.764210,17.150671,39.957198,20.368499,36.412788,19.423401,34.952577,...,21.535751,39.741507,18.376689,40.370786,-0.636235,768.945232,93.468778,3.599316,48.901578,-1.036065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4929,59.467450,0.777253,24.485158,47.250603,22.892884,48.801468,26.272932,43.499272,24.400779,44.444640,...,25.736635,49.762088,23.412199,47.736994,11.678544,744.925444,98.785571,1.623487,58.194650,12.035595
4930,58.941493,0.780822,20.581104,40.746583,17.461279,45.169743,21.443705,38.277698,20.295528,36.482143,...,22.199121,41.552381,19.275698,39.648044,3.974215,763.654684,88.383353,3.676556,55.844162,2.518998
4931,78.144250,7.731828,21.240040,37.921562,20.120400,37.666955,21.150648,37.849118,20.814991,36.310247,...,21.550008,38.176926,18.921595,40.789138,6.577102,748.177254,76.495594,5.525237,39.876868,2.166772
4932,77.310668,8.125201,22.360369,39.968537,21.819112,38.742542,21.606685,41.901135,20.910382,42.514454,...,21.322319,45.619732,17.996486,44.078257,2.515017,759.823470,95.414514,3.514581,34.160163,1.889147


In [12]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)

In [13]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('Appliances(t)', 65.486, 26.569, 0.578, 0.228)
Results: ('lights(t)', 4.876, 2.735, 0.606, 4764085368086506.0)
Results: ('T1(t)', 0.402, 0.32, 0.936, 0.015)
Results: ('RH_1(t)', 0.637, 0.422, 0.974, 0.01)
Results: ('T2(t)', 0.466, 0.384, 0.954, 0.019)
Results: ('RH_2(t)', 0.484, 0.368, 0.985, 0.009)
Results: ('T3(t)', 0.47, 0.37, 0.944, 0.017)
Results: ('RH_3(t)', 0.522, 0.394, 0.974, 0.01)
Results: ('T4(t)', 0.481, 0.386, 0.943, 0.019)
Results: ('RH_4(t)', 0.503, 0.401, 0.987, 0.01)
Results: ('T5(t)', 0.462, 0.353, 0.936, 0.018)
Results: ('RH_5(t)', 2.275, 0.637, 0.937, 0.011)
Results: ('T6(t)', 0.511, 0.397, 0.993, 1214408206629.366)
Results: ('RH_6(t)', 1.358, 0.813, 0.998, 0.077)
Results: ('T7(t)', 0.443, 0.358, 0.955, 0.018)
Results: ('RH_7(t)', 0.511, 0.425, 0.99, 0.012)
Results: ('T8(t)', 0.483, 0.387, 0.938, 0.018)
Results: ('RH_8(t)', 0.498, 0.403, 0.991, 0.009)
Results: ('T9(t)', 0.39, 0.31, 0.962, 0.016)
Results: ('RH_9(t)', 0.499, 0.397, 0.985, 0.01)
Results: ('T_

,variable,rmse,mae,r2,mape
0,Appliances(t),65.486,26.569,0.578,2.280000e-01
1,lights(t),4.876,2.735,0.606,4.764085e+15
2,T1(t),0.402,0.320,0.936,1.500000e-02
3,RH_1(t),0.637,0.422,0.974,1.000000e-02
4,T2(t),0.466,0.384,0.954,1.900000e-02
5,RH_2(t),0.484,0.368,0.985,9.000000e-03
6,T3(t),0.470,0.370,0.944,1.700000e-02
7,RH_3(t),0.522,0.394,0.974,1.000000e-02
8,T4(t),0.481,0.386,0.943,1.900000e-02
9,RH_4(t),0.503,0.401,0.987,1.000000e-02


## MinMaxScaler

In [81]:
#data_red = data.loc[:10000,:]
x,y=data.loc[:,:],data.loc[:,:]

In [82]:
from sklearn.preprocessing import MinMaxScaler
scalerx = MinMaxScaler()
scalery=MinMaxScaler()

In [83]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Get the number of inputs and outputs from the dataset
df_trainx=x_train.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
df_trainx_scaled = scalerx.fit_transform(df_trainx)
df_testx=x_test.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
df_testx_scaled = scalerx.transform(df_testx)
df_trainy=y_train.loc[:,'Appliances(t)':'Tdewpoint(t)']
df_trainy_scaled=scalery.fit_transform(df_trainy)

df_original=y_test.loc[:,'Appliances(t)':'Tdewpoint(t)']

n_inputs, n_outputs = df_trainx.shape[1], df_trainy.shape[1]

In [84]:
from scipy import stats
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
# Define the hyperparameter configuration space
hyperparameters = dict(
                     estimator__C=stats.uniform(0,50),
                     estimator__kernel=['poly', 'rbf','sigmoid'],
                     estimator__epsilon=stats.uniform(0,1))
clf=SVR()
model=MultiOutputRegressor(clf)
gsc = RandomizedSearchCV(model,hyperparameters,cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1,n_iter=10)

hyperparameters_tuning_nor = gsc.fit(df_trainx_scaled, df_trainy_scaled)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning_nor.best_estimator_

Best Parameters = {'estimator__C': 18.291137504308498, 'estimator__epsilon': 0.7817324250849084, 'estimator__kernel': 'poly'}


In [86]:
hyperparameters_tuning_nor.best_estimator_

MultiOutputRegressor(estimator=SVR(C=5.7642040130783005,
                                   epsilon=0.07067015766896201))

In [25]:

x,y=data.loc[:,:],data.loc[:,:]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Get the number of inputs and outputs from the dataset
df_trainx=x_train.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
df_trainx_scaled = scalerx.fit_transform(df_trainx)
df_testx=x_test.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
df_testx_scaled = scalerx.transform(df_testx)
df_trainy=y_train.loc[:,'Appliances(t)':'Tdewpoint(t)']
df_trainy_scaled=scalery.fit_transform(df_trainy)

df_original=y_test.loc[:,'Appliances(t)':'Tdewpoint(t)']

In [26]:
y_p=tuned_model.predict(df_testx_scaled)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,Appliances(t),lights(t),T1(t),RH_1(t),T2(t),RH_2(t),T3(t),RH_3(t),T4(t),RH_4(t),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
0,0.120047,0.126805,0.451675,0.285318,0.118047,0.574457,0.267046,0.354568,0.441238,0.288305,...,0.573762,0.338101,0.540018,0.403072,0.233551,0.914058,0.850166,0.137836,0.829621,0.300873
1,0.082108,0.004497,0.465025,0.744686,0.499007,0.757030,0.442416,0.956318,0.493514,0.502099,...,0.391618,0.614695,0.210448,0.538737,0.312899,0.597666,0.838038,0.250804,0.504440,0.361308
2,0.070419,0.055071,0.467828,0.257318,0.113211,0.574612,0.378622,0.277537,0.492602,0.236989,...,0.493785,0.306715,0.540657,0.346951,0.174311,0.848385,0.770999,0.120943,0.902854,0.197146
3,0.160620,0.129549,0.540876,0.462239,0.568879,0.447704,0.478776,0.397892,0.671518,0.347189,...,0.438317,0.318618,0.573493,0.365029,0.531788,0.846094,0.558429,0.297019,0.557033,0.467956
4,0.065109,0.104488,0.356925,0.281259,0.047051,0.589909,0.226771,0.305638,0.395546,0.304570,...,0.482686,0.318654,0.496105,0.462300,0.172814,0.930289,0.914658,0.231301,0.748938,0.258300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4929,-0.005561,0.180537,0.676182,0.536540,0.622880,0.755971,0.677345,0.565276,0.729185,0.665242,...,0.729855,0.613997,0.667981,0.605480,0.535055,0.445751,0.952164,0.065496,0.682005,0.748730
4930,0.096260,0.142665,0.446022,0.434322,0.129801,0.754628,0.297391,0.414266,0.512047,0.316213,...,0.491830,0.357457,0.526428,0.382994,0.303451,0.806253,0.874793,0.254714,0.757470,0.440379
4931,0.151403,0.218440,0.433552,0.329161,0.292093,0.493296,0.258050,0.378569,0.594557,0.342826,...,0.441740,0.255552,0.470633,0.510288,0.401744,0.454313,0.707574,0.403639,0.597088,0.443841
4932,0.143374,0.064743,0.554778,0.385935,0.533482,0.527314,0.383013,0.639814,0.619932,0.628603,...,0.396943,0.546747,0.356569,0.640747,0.297731,0.706638,0.907988,0.205108,0.458285,0.373693


In [27]:
y_p_inv = scalery.inverse_transform(y_p)
columns=df_original.columns
y_p_inv=pd.DataFrame(y_p_inv,columns=columns)
y_p_inv

,Appliances(t),lights(t),T1(t),RH_1(t),T2(t),RH_2(t),T3(t),RH_3(t),T4(t),RH_4(t),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
0,138.449969,8.876345,21.067362,37.390822,17.723928,40.892956,20.414167,36.327232,19.997738,34.958919,...,22.574063,39.465789,20.079569,38.904887,2.263448,768.604492,88.612589,1.860787,54.925341,0.019206
1,97.855132,0.314825,21.193788,54.082730,22.964666,47.385853,22.524925,49.158554,20.578001,39.804714,...,20.584444,47.536811,16.912405,42.182554,4.731171,754.999628,87.690859,3.385849,33.788582,1.348767
2,85.348794,3.854955,21.220333,36.373400,17.657411,40.898436,21.757100,34.684677,20.567878,33.795802,...,21.700448,38.549952,20.085716,37.549011,0.421073,765.780562,82.595940,1.632731,59.685534,-2.262778
3,181.863471,9.068448,21.912099,43.819543,23.925883,36.385177,22.962553,37.251040,22.553848,36.293575,...,21.094546,38.897269,20.401266,37.985777,11.538620,765.682049,66.440602,4.009755,37.207147,3.695024
4,79.666241,7.314130,20.170078,37.243339,16.747267,41.442476,19.929417,35.283890,19.490565,35.327587,...,21.579205,38.898318,19.657572,40.335840,0.374505,769.302448,93.514003,3.122565,49.680957,-0.917400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4929,4.049598,12.637591,23.193447,46.519413,24.668756,47.348175,25.352523,40.820242,23.193957,43.502480,...,24.279121,47.516426,21.309297,43.795065,11.640211,748.467300,96.364466,0.884202,45.330309,9.872052
4930,112.998094,9.986557,21.013831,42.805141,17.885630,47.300406,20.779400,37.600202,20.783721,35.591478,...,21.679094,40.030609,19.948972,38.419796,4.437333,763.968887,90.484288,3.438635,50.235551,3.088344
4931,172.001417,15.290776,20.895737,38.983930,20.118229,38.006580,20.305894,36.839011,21.699587,36.194691,...,21.131940,37.056994,19.412783,41.495223,7.494249,748.835471,77.775634,5.449126,39.810702,3.164508
4932,163.410263,4.532000,22.043744,41.046925,23.438929,39.216367,21.809949,42.409643,21.981248,42.672019,...,20.642605,45.554072,18.316628,44.647110,4.259434,759.685438,93.007118,2.768954,30.788529,1.621250


In [28]:
df_original

,Appliances(t),lights(t),T1(t),RH_1(t),T2(t),RH_2(t),T3(t),RH_3(t),T4(t),RH_4(t),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
8981,40.0,0.0,20.856667,35.363333,17.70,39.200,20.290000,36.900000,19.700000,34.200000,...,23.043750,40.251250,19.290000,39.090000,1.783333,768.850000,88.166667,2.666667,63.000000,0.000000
2755,90.0,10.0,21.890000,50.890000,21.29,45.060,21.566667,48.524583,20.356667,40.933333,...,20.600000,47.416667,16.290000,44.163333,2.733333,754.550000,89.500000,3.166667,27.333333,1.166667
9133,40.0,0.0,21.390000,35.500000,17.60,40.530,21.600000,35.200000,20.260000,33.700000,...,22.100000,39.700000,19.390000,38.372500,0.083333,766.166667,83.500000,1.833333,64.666667,-2.433333
14359,50.0,0.0,21.390000,41.033333,23.89,34.840,22.033333,36.933333,22.390000,35.236000,...,21.200000,38.090000,20.290000,38.560000,8.850000,767.800000,70.833333,5.166667,40.000000,3.716667
8876,70.0,0.0,19.890000,35.200000,16.50,40.090,20.000000,36.400000,19.230000,35.156667,...,22.100000,39.663333,19.000000,40.700000,-0.333333,769.700000,92.666667,4.000000,50.666667,-1.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17496,60.0,0.0,24.790000,47.090000,22.70,49.290,26.533333,44.500000,25.142857,44.212857,...,25.790000,49.590000,23.968571,47.942857,12.300000,744.700000,99.000000,1.000000,61.000000,12.100000
9283,50.0,0.0,20.790000,40.700000,17.60,46.790,20.790000,38.230000,20.566667,35.700000,...,22.290000,41.760000,19.566667,39.200000,4.216667,764.300000,88.500000,3.833333,55.000000,2.500000
7472,60.0,0.0,21.085714,37.470000,19.89,37.270,20.200000,38.000000,22.075000,35.785000,...,21.264286,37.264286,18.600000,41.700000,6.566667,748.400000,75.333333,5.333333,40.000000,2.433333
611,80.0,10.0,23.200000,39.363333,22.39,38.260,21.133333,41.863333,21.700000,42.900000,...,20.790000,45.096667,17.600000,44.960000,2.616667,760.500000,95.166667,3.166667,37.166667,1.933333


In [29]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p_inv[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)
 

In [30]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('Appliances(t)', 126.924, 82.784, -0.586, 1.108)
Results: ('lights(t)', 7.881, 6.026, -0.03, 1.7736545632452994e+16)
Results: ('T1(t)', 0.505, 0.363, 0.899, 0.016)
Results: ('RH_1(t)', 2.726, 2.469, 0.53, 0.061)
Results: ('T2(t)', 0.669, 0.537, 0.905, 0.026)
Results: ('RH_2(t)', 1.505, 1.239, 0.859, 0.031)
Results: ('T3(t)', 0.501, 0.347, 0.937, 0.015)
Results: ('RH_3(t)', 0.964, 0.678, 0.912, 0.017)
Results: ('T4(t)', 0.544, 0.34, 0.927, 0.016)
Results: ('RH_4(t)', 1.121, 0.754, 0.934, 0.018)
Results: ('T5(t)', 0.785, 0.523, 0.814, 0.025)
Results: ('RH_5(t)', 3.241, 1.828, 0.873, 0.034)
Results: ('T6(t)', 2.084, 1.519, 0.88, 1530521665996.092)
Results: ('RH_6(t)', 15.859, 10.809, 0.736, 2.575)
Results: ('T7(t)', 0.793, 0.455, 0.856, 0.021)
Results: ('RH_7(t)', 1.484, 0.922, 0.915, 0.025)
Results: ('T8(t)', 0.55, 0.424, 0.92, 0.019)
Results: ('RH_8(t)', 1.066, 0.795, 0.958, 0.018)
Results: ('T9(t)', 0.76, 0.551, 0.855, 0.028)
Results: ('RH_9(t)', 1.042, 0.704, 0.937, 0.016)
R

,variable,rmse,mae,r2,mape
0,Appliances(t),126.924,82.784,-0.586,1.108000e+00
1,lights(t),7.881,6.026,-0.030,1.773655e+16
2,T1(t),0.505,0.363,0.899,1.600000e-02
3,RH_1(t),2.726,2.469,0.530,6.100000e-02
4,T2(t),0.669,0.537,0.905,2.600000e-02
5,RH_2(t),1.505,1.239,0.859,3.100000e-02
6,T3(t),0.501,0.347,0.937,1.500000e-02
7,RH_3(t),0.964,0.678,0.912,1.700000e-02
8,T4(t),0.544,0.340,0.927,1.600000e-02
9,RH_4(t),1.121,0.754,0.934,1.800000e-02
